In [8]:
# imports
import keras_tuner as kt
from keras.layers import Dense
from keras.layers import Dropout
from keras.optimizers import Adam
from keras.models import Sequential
from keras.callbacks import EarlyStopping
from nnTrain import getSplitData
from selectionPlots import findAllFilesInPath

In [3]:
def model_builder(hp):
  model = Sequential()

  for i in range(hp.Int("num_layers", 1, 6)): # Variable number of hidden layers
    model.add(Dense(units = hp.Int(f"units_{i}", min_value = 4, max_value = 32, step = 2),
                    activation = hp.Choice("activation", ["relu", "tanh", "elu"]))) # Hidden layer

  if hp.Boolean("dropout"): # Possible dropout layer to prevent overfitting
    model.add(Dropout(rate = hp.Float("dropout_rate", min_value = 0.0, max_value = 0.5, step = 0.1)))

  model.add(Dense(1, activation = "sigmoid")) # Output layer

  learning_rate = hp.Float("lr", min_value = 1e-4, max_value = 1e-2, sampling = "log")

  # Compile the model
  model.compile(loss = "binary_crossentropy", optimizer = Adam(learning_rate = learning_rate),
                metrics = ["accuracy"])

  return model

In [4]:
sampleNames = findAllFilesInPath("*.root", "nTupleGroups/")
nTupleSamples = dict.fromkeys(sampleNames, 0)
nTupleSamples["nTupleGroups/signalGroup.root"] = 1

# Tuple of variables to get from each file
variables = ["tauPtSum", "zMassSum", "metPt", "deltaRll", "deltaRtt", "deltaRttll", "deltaEtall", "deltaEtatt",
             "nJets", "deltaPhill", "deltaPhitt", "deltaPhilltt", "mmc"]

cut = "2lep"
X_train, X_test, y_train, y_test = getSplitData(nTupleSamples, variables, cut, 0)

In [5]:
# Create the tuner
tuner = kt.Hyperband(model_builder,
                      objective = "val_accuracy",
                      max_epochs = 30,
                      factor = 3,
                      directory = "hyperTuning",
                      project_name = "hyperTuning" + cut,
                      overwrite = True)

stop_early = EarlyStopping(monitor = "val_loss", patience = 5)

2023-02-15 17:35:25.694318: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-02-15 17:35:25.694415: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/george/root/lib
2023-02-15 17:35:25.694471: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/george/root/lib
2023-02-15 17:35:25.694508: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open sha

In [6]:
# 0.25*0.8 = 0.2 to get 20% of the data for validation
tuner.search(X_train, y_train, epochs = 10, validation_split = 0.25, callbacks = [stop_early])

Trial 49 Complete [00h 00m 04s]
val_accuracy: 0.8292446732521057

Best val_accuracy So Far: 0.8308586478233337
Total elapsed time: 00h 01m 23s

Search: Running Trial #50

Value             |Best Value So Far |Hyperparameter
relu              |relu              |activation
5                 |5                 |num_layers
22                |24                |units_0
0.0034366         |0.0052679         |lr
16                |26                |units_1
26                |28                |units_2
14                |14                |units_3
26                |20                |units_4
18                |30                |units_5
10                |10                |tuner/epochs
4                 |4                 |tuner/initial_epoch
3                 |3                 |tuner/bracket
2                 |2                 |tuner/round
0038              |0041              |tuner/trial_id

Epoch 5/10
291/291 [==============================] - 1s 3ms/step - loss: 0.3701 - accuracy: 0.8

KeyboardInterrupt: 

In [ ]:
# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials = 1)[0]

print("The hyperparameter search is complete. The optimal number of layers was:", best_hps.get("num_layers"))

for i in range(best_hps.get("num_layers") + 1):
  print("layer", i, ":", best_hps.get(f"units_{i}"), best_hps.get("activation"))

print("Learning rate:", best_hps.get("lr"))

NameError: name 'tuner' is not defined

In [ ]:
# Build the model with the optimal hyperparameters and train it on the data for 50 epochs
model = tuner.hypermodel.build(best_hps)
history = model.fit(X_train, y_train, epochs = 50, validation_split = 0.25, callbacks = [stop_early])

# Find the best number of epochs to train for
valAccPerEpoch = history.history["val_accuracy"]
bestEpoch = valAccPerEpoch.index(max(valAccPerEpoch)) + 1
print("Best epoch:", bestEpoch)

In [ ]:
# Train the model with the optimal number of epochs
model = tuner.hypermodel.build(best_hps)
history = model.fit(X_train, y_train, epochs = bestEpoch, validation_split = 0.25)

In [ ]:
# Save the model
model.save("nnModels/hyperTuned" + cut + ".h5")